In [7]:
#importing libraries
import pandas as pd
import numpy as np
import sqlite3
import uuid
import pytz
from datetime import datetime, timedelta, time
import os


In [8]:
# load data
df_status = pd.read_csv('store_status.csv')
df_hours = pd.read_csv('menu_hours.csv')
df_timezones = pd.read_csv('timezones.csv')

print("Data loaded:")
print(f"Status records: {len(df_status)}")
print(f"Business hours records: {len(df_hours)}")
print(f"Timezone records: {len(df_timezones)}")

display(df_status.head())
display(df_hours.head())
display(df_timezones.head())


Data loaded:
Status records: 1849837
Business hours records: 35457
Timezone records: 4559


,store_id,status,timestamp_utc
0,b5d0a65d-6d54-47aa-95e9-9312f0353326,active,2024-10-03 23:33:20.412748 UTC
1,a792089b-e23d-435f-bc18-113b7cc95e11,active,2024-10-03 23:33:36.781143 UTC
2,7a242d0e-309c-4915-9755-e9019d69108d,active,2024-10-03 23:33:37.536328 UTC
3,ca793240-b974-4551-ba0b-649d1a52956c,active,2024-10-03 23:33:56.752347 UTC
4,3a2313be-27d9-429f-9906-ccd142d9906c,active,2024-10-03 23:34:04.138852 UTC


,store_id,dayOfWeek,start_time_local,end_time_local
0,d84a4552-3668-4075-ad1d-16840294f818,1,00:00:00,00:01:00
1,d84a4552-3668-4075-ad1d-16840294f818,3,00:00:00,00:01:00
2,d84a4552-3668-4075-ad1d-16840294f818,4,00:00:00,00:01:00
3,d84a4552-3668-4075-ad1d-16840294f818,0,00:00:00,00:01:00
4,d84a4552-3668-4075-ad1d-16840294f818,5,00:00:00,00:01:00


,store_id,timezone_str
0,f6e4d870-a273-49cf-be4d-1bf529e26ff3,America/Boise
1,7fa112c6-fd3a-4ff6-aadb-4b55f2e74048,America/Boise
2,273c2570-da54-4526-ab33-175e4d7a1609,America/Boise
3,bad3c0e6-ad67-40b4-aca8-ce1046b25d68,America/Boise
4,0df89218-fde7-4fc6-83da-1c2fc7b480a0,America/Boise


In [9]:
#seting up database
con = sqlite3.connect("store_monitoring.db")
df_status.to_sql('store_status', con, if_exists='replace', index=False)
df_hours.to_sql('menu_hours', con, if_exists='replace', index=False)
df_timezones.to_sql('timezones', con, if_exists='replace', index=False)

4559

In [10]:
#helper functions

def get_store_timezone(store_id, tz_df):
    try:
        tz_str = tz_df[tz_df['store_id'] == store_id]['timezone_str'].iloc[0]
        return pytz.timezone(tz_str)
    except (IndexError, KeyError):
        return pytz.timezone('America/Chicago')

def get_store_hours(store_id, day_of_week, hours_df):
    store_hours = hours_df[
        (hours_df['store_id'] == store_id) & 
        (hours_df['dayOfWeek'] == day_of_week)
    ]
    
    if len(store_hours) == 0:
        return [(time(0, 0, 0), time(23, 59, 59))]
    
    hours_list = []
    for _, row in store_hours.iterrows():
        start = datetime.strptime(row['start_time_local'], "%H:%M:%S").time()
        end = datetime.strptime(row['end_time_local'], "%H:%M:%S").time()
        hours_list.append((start, end))
    
    return hours_list

In [11]:
#interpolation logic

def calculate_uptime_downtime(store_id, window_start, window_end, hours_df, tz_df, status_df):
    #ensure timezone-naive timestamps
    if hasattr(window_start, 'tz'):
        window_start = window_start.tz_localize(None) if window_start.tz is None else window_start.tz_convert('UTC').tz_localize(None)
    if hasattr(window_end, 'tz'):
        window_end = window_end.tz_localize(None) if window_end.tz is None else window_end.tz_convert('UTC').tz_localize(None)
    
    store_tz = get_store_timezone(store_id, tz_df)
    
    #relevant status data
    store_data = status_df[status_df['store_id'] == store_id].copy()
    store_data['timestamp_utc'] = pd.to_datetime(store_data['timestamp_utc']).dt.tz_localize(None)
    
    relevant_data = store_data[
        (store_data['timestamp_utc'] >= window_start) & 
        (store_data['timestamp_utc'] <= window_end)
    ].sort_values('timestamp_utc')
    
    if len(relevant_data) == 0:
        return 0, 0
    
    total_uptime_minutes = 0
    total_downtime_minutes = 0
    
    #process each day in the window
    current_day = window_start.date()
    end_day = window_end.date()
    
    while current_day <= end_day:
        day_start_utc = datetime.combine(current_day, time.min)
        day_start_local = pytz.utc.localize(day_start_utc).astimezone(store_tz)
        day_of_week = day_start_local.weekday()
        
        business_intervals = get_store_hours(store_id, day_of_week, hours_df)
        
        for start_time, end_time in business_intervals:
            interval_start_local = datetime.combine(current_day, start_time)
            interval_end_local = datetime.combine(current_day, end_time)
            
            if interval_end_local <= interval_start_local:
                interval_end_local += timedelta(days=1)
            
            try:
                interval_start_utc = store_tz.localize(interval_start_local).astimezone(pytz.utc).replace(tzinfo=None)
                interval_end_utc = store_tz.localize(interval_end_local).astimezone(pytz.utc).replace(tzinfo=None)
            except:
                continue
            
            analysis_start = max(interval_start_utc, window_start)
            analysis_end = min(interval_end_utc, window_end)
            
            if analysis_start >= analysis_end:
                continue
            
            interval_observations = relevant_data[
                (relevant_data['timestamp_utc'] >= analysis_start) &
                (relevant_data['timestamp_utc'] <= analysis_end)
            ]
            
            #finding previous status for interpolation
            previous_status = 'active'
            before_interval = relevant_data[relevant_data['timestamp_utc'] <= analysis_start]
            if len(before_interval) > 0:
                previous_status = before_interval.iloc[-1]['status']
            
            #build timeline
            timeline = [(analysis_start, previous_status)]
            for _, obs in interval_observations.iterrows():
                timeline.append((obs['timestamp_utc'], obs['status']))
            timeline.append((analysis_end, None))
            
            #calculating durations
            for i in range(len(timeline) - 1):
                period_start, status = timeline[i]
                period_end, _ = timeline[i + 1]
                
                duration_minutes = (period_end - period_start).total_seconds() / 60
                
                if status == 'active':
                    total_uptime_minutes += duration_minutes
                else:
                    total_downtime_minutes += duration_minutes
        
        current_day += timedelta(days=1)
    
    return total_uptime_minutes, total_downtime_minutes

In [12]:
#test samples logic
def test_sample_stores():
    df_status['timestamp_utc'] = pd.to_datetime(df_status['timestamp_utc'])
    report_time = df_status['timestamp_utc'].max()
    if hasattr(report_time, 'tz_localize'):
        report_time = report_time.tz_localize(None)
    
    test_stores = df_status['store_id'].unique()[:3]
    
    for store_id in test_stores:
        hour_start = report_time - pd.Timedelta(hours=1)
        uptime_min, downtime_min = calculate_uptime_downtime(
            store_id, hour_start, report_time, df_hours, df_timezones, df_status
        )
        print(f"Store {store_id} - Last hour: {uptime_min:.1f}min up, {downtime_min:.1f}min down")

test_sample_stores()

Store b5d0a65d-6d54-47aa-95e9-9312f0353326 - Last hour: 60.0min up, 0.0min down
Store a792089b-e23d-435f-bc18-113b7cc95e11 - Last hour: 60.0min up, 0.0min down
Store 7a242d0e-309c-4915-9755-e9019d69108d - Last hour: 60.0min up, 0.0min down


In [13]:
#generate report 
def generate_complete_report():
    all_stores = df_status['store_id'].unique()
    current_time = df_status['timestamp_utc'].max()
    if hasattr(current_time, 'tz_localize'):
        current_time = current_time.tz_localize(None)
    
    one_hour_ago = current_time - pd.Timedelta(hours=1)
    one_day_ago = current_time - pd.Timedelta(days=1)
    one_week_ago = current_time - pd.Timedelta(days=7)
    
    report_data = []
    
    for i, store_id in enumerate(all_stores):
        if i % 100 == 0:
            print(f"Processing {i}/{len(all_stores)}")
        
        try:
            uptime_hour, downtime_hour = calculate_uptime_downtime(
                store_id, one_hour_ago, current_time, df_hours, df_timezones, df_status
            )
            
            uptime_day, downtime_day = calculate_uptime_downtime(
                store_id, one_day_ago, current_time, df_hours, df_timezones, df_status
            )
            
            uptime_week, downtime_week = calculate_uptime_downtime(
                store_id, one_week_ago, current_time, df_hours, df_timezones, df_status
            )
            
            report_row = {
                'store_id': store_id,
                'uptime_last_hour(in minutes)': round(uptime_hour, 2),
                'uptime_last_day(in hours)': round(uptime_day / 60, 2),
                'uptime_last_week(in hours)': round(uptime_week / 60, 2),
                'downtime_last_hour(in minutes)': round(downtime_hour, 2),
                'downtime_last_day(in hours)': round(downtime_day / 60, 2),
                'downtime_last_week(in hours)': round(downtime_week / 60, 2)
            }
            
            report_data.append(report_row)
            
        except Exception as e:
            report_row = {
                'store_id': store_id,
                'uptime_last_hour(in minutes)': 0,
                'uptime_last_day(in hours)': 0,
                'uptime_last_week(in hours)': 0,
                'downtime_last_hour(in minutes)': 0,
                'downtime_last_day(in hours)': 0,
                'downtime_last_week(in hours)': 0
            }
            report_data.append(report_row)
    
    report_df = pd.DataFrame(report_data)
    return report_df

In [14]:
#API integration 
report_storage = {}

def trigger_report():
    report_id = str(uuid.uuid4())
    print(f"Report ID: {report_id}")
    
    report_storage[report_id] = {'status': 'Running'}
    
    try:
        report_df = generate_complete_report()
        csv_filename = f"report_{report_id}.csv"
        report_df.to_csv(csv_filename, index=False)
        
        report_storage[report_id] = {
            'status': 'Complete',
            'data': report_df,
            'filename': csv_filename
        }
        
        print(f"Report completed: {csv_filename}")
        
    except Exception as e:
        report_storage[report_id] = {'status': 'Failed', 'error': str(e)}
    
    return report_id

def get_report(report_id):
    if report_id not in report_storage:
        return "Invalid report_id"
    
    report_info = report_storage[report_id]
    status = report_info['status']
    
    if status == 'Running':
        return "Running"
    elif status == 'Complete':
        return "Complete", report_info['data']
    elif status == 'Failed':
        return f"Failed: {report_info.get('error', 'Unknown error')}"

In [15]:
#execution of the flow 

report_id = trigger_report()
result = get_report(report_id)

if isinstance(result, tuple) and result[0] == "Complete":
    status, report_df = result
    print(f"\nReport Status: {status}")
    print(f"Total stores processed: {len(report_df)}")
    print("\nSample results:")
    print(report_df.head())
    
    # verify schema
    expected_columns = [
        'store_id',
        'uptime_last_hour(in minutes)',
        'uptime_last_day(in hours)', 
        'uptime_last_week(in hours)',
        'downtime_last_hour(in minutes)',
        'downtime_last_day(in hours)',
        'downtime_last_week(in hours)'
    ]
    
    print(f"\nSchema verification: {report_df.columns.tolist() == expected_columns}")
else:
    print(f"Result: {result}")

Report ID: 3c789b8a-bf0c-4580-8e95-63f2b41f89c2
Processing 0/3678
Processing 100/3678
Processing 200/3678
Processing 300/3678
Processing 400/3678
Processing 500/3678
Processing 600/3678
Processing 700/3678
Processing 800/3678
Processing 900/3678
Processing 1000/3678
Processing 1100/3678
Processing 1200/3678
Processing 1300/3678
Processing 1400/3678
Processing 1500/3678
Processing 1600/3678
Processing 1700/3678
Processing 1800/3678
Processing 1900/3678
Processing 2000/3678
Processing 2100/3678
Processing 2200/3678
Processing 2300/3678
Processing 2400/3678
Processing 2500/3678
Processing 2600/3678
Processing 2700/3678
Processing 2800/3678
Processing 2900/3678
Processing 3000/3678
Processing 3100/3678
Processing 3200/3678
Processing 3300/3678
Processing 3400/3678
Processing 3500/3678
Processing 3600/3678
Report completed: report_3c789b8a-bf0c-4580-8e95-63f2b41f89c2.csv

Report Status: Complete
Total stores processed: 3678

Sample results:
                               store_id  uptime_la